In [95]:
import numpy
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Data Understanding
In what follows, we try to take a preliminary look at the data and understand what information we have available.  

### 1.1. Data from the Box Office Mojo

In [96]:
# import data from Box Office Mojo
df_bom = pd.read_csv("./unzipped_data/bom.movie_gross.csv")
print("summary of the data from Box Office Mojo:")
print(df_bom.info())
df_bom.head()

summary of the data from Box Office Mojo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [97]:
# what percentage of data is missing from the domestic gross & foreign gross columns?
print(
    "% missing data from the domestic gross column:",
    (1 - len(df_bom.dropna(subset=["domestic_gross"])) / len(df_bom)) * 100,
)
print(
    "% missing data from the foreign gross column:",
    (1 - len(df_bom.dropna(subset=["foreign_gross"])) / len(df_bom)) * 100,
)
print(
    "% missing data from the foreign domestic & gross columns combined:",
    (1 - len(df_bom.dropna(subset=["domestic_gross", "foreign_gross"])) / len(df_bom))
    * 100,
)

% missing data from the domestic gross column: 0.8266902863891312
% missing data from the foreign gross column: 39.85828166519043
% missing data from the foreign domestic & gross columns combined: 40.68497195157958


In [98]:
df_bom["foreign_gross"] = (
    df_bom["foreign_gross"].str.replace(",", "", regex=False).astype(float)
)  # convert the foreign gross data type to float
df_bom.dropna(subset=["domestic_gross", "foreign_gross"])[
    ["domestic_gross", "foreign_gross"]
].describe()

,domestic_gross,foreign_gross
count,2.009000e+03,2.009000e+03
mean,4.697311e+07,7.571822e+07
std,8.159966e+07,1.381296e+08
min,4.000000e+02,6.000000e+02
25%,6.650000e+05,3.900000e+06
50%,1.650000e+07,1.930000e+07
75%,5.600000e+07,7.590000e+07
max,9.367000e+08,9.605000e+08


**Observations:**
1. The data from Box Office Mojo has 3387 entries and it contains information on the movie name, studio, and the year it was produced. There's also information on the **domestic gross** and **foregin gross** revenue of the movies. The former is the total revenue a movie earns in its home country (in this case, United States and Canada) while the latter refers to the total revenue a movie earns outside of its home country. This information will come in handy for determining the **Return on Investment** metric.
2. The data type for foreign gross is object (text). We need to change it to float. 
3. We also notice that several entries are missing from the domestic gross (less than 1%) and foreign gross (~40%) columns. This data will need cleaning (including the removal of NaNs) before it can be used in our analysis. 

### 1.2. Data from IMDB

In [99]:
# import data from IMDB
conn = sqlite3.connect("./unzipped_data/im.db")  # open up a connection
cursor = conn.cursor()
# table name query
table_name_query = (
    """SELECT name AS table_name FROM sqlite_master WHERE type='table';"""
)
imdb_sql_table_names = pd.read_sql(table_name_query, conn)
imdb_sql_table_names.head(10)

,table_name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


Let's explore each table and understand what useful information they might contain. Below we define a function that queries each table to show all its columns. We will reuse this function to take a peek at the data in all tables.

In [100]:
def show_table_contents(table_name):
    read_table_query = f"""SELECT *
                        FROM {table_name}"""
    print(f"This is the content of {table_name}")
    print(pd.read_sql(read_table_query, conn).head());

#### 1.2.1. IMDB Data: Table Contents

In [101]:
[
    show_table_contents(table_name)
    for table_name in list(imdb_sql_table_names["table_name"])
];

This is the content of movie_basics
    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  
0        2013            175.0    Action,Crime,Drama  
1        2019            114.0       Biography,Drama  
2        2018            122.0                 Drama  
3        2018              NaN          Comedy,Drama  
4        2017             80.0  Comedy,Drama,Fantasy  
This is the content of directors
    movie_id  person_id
0  tt0285252  nm0899854
1  tt0462036  nm1940585
2  tt0835418  nm0151540
3  tt0835418  nm0151540
4  tt0878654  nm00

**Observations:** }This database has some useful information about **movie ratings** and **number of votes** as well as **directors** and **actors**. Since we would like to focus on a fiscal measure such as ROI, this data does not seem particularly necessary to include in our analysis. However, we may use this data at the end to study correlations between high ratings and other factors mentioned above. 

### 1.3. Data from Rotten Tomatoes

In [102]:
# import data on movie reviews from rotton tomatoes
df_rt_movie_info = pd.read_csv("./unzipped_data/rt.movie_info.tsv", delimiter="\t")
print("Summary of the data on movie info from Rotton Tomatoes:")
print(df_rt_movie_info.info())
df_rt_movie_info.head()

Summary of the data on movie info from Rotton Tomatoes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB
None


,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


The data from Rotten Tomatoes contains information on the **total revenue generate by a movie** (box_office). It also contains other interesting information such as **genre** and **rating**. What is striking though is that the box_office field has a high percentage of missing data (roughly 78%). It's not clear how useful this data will be in terms of considering the revenue. The other problem is that the name of the movies is not listed so there is no way to cross-reference this table with other tables. 

In [103]:
df_rt_reviews = pd.read_csv("./unzipped_data/rt.reviews.tsv", sep="\t", encoding="mbcs")
print(df_rt_reviews.info())
df_rt_reviews.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB
None


,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


This table lists the full reviews people left for movies. The data regarding movie ratings can be combined with that from IMDB if we do an analysis that looks at whether higher ratings correlate with better fiscal outcomes. Further down the line, it may also be possible to train a model that can classify the reviews as positive or negative. Then use that to do further studies on how different movies were generally received. We abandon this idea for now. 

### 1.4. Data from TheMovieDB

In [104]:
# import data from the movie db
df_movie_db = pd.read_csv("./unzipped_data/tmdb.movies.csv", index_col=0)
df_movie_db.head()

,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


This table has information on movie ratings (**vote_average**) and **popularity**. The popularity column does not have a specific range and is determined based on several factors including the recentness of the release and the number of votes. This table also includes information on the **genres**. However, they're listed as ids. If we intend to use this information, we will need to first map the ids to the actual genres using a mapping that TMDB provides itself. This table could also be joined with the table from IMDB and Rotton Tomatoes containing similar information. We shelf these ideas for now since we have decided to focus on a fiscal measure and will return to them later. 

### 1.5. Data from TheNumbers

In [94]:
# import data from The Numbers
df_movie_budgets = pd.read_csv("./unzipped_data/tn.movie_budgets.csv")
df_movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


This table is also of interest to us for the main analysis since it has information on the **production budget** and the **domestic gross** as well as the **worldwide gross**. We could join this data frame with the one we got from Box Office Mojo to study the fiscal measure of ROI and assess the success of movies.  